In [ ]:
# Required Imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, Button, VBox, Output
from IPython.display import display, clear_output
import time

University facilities data

In [ ]:
facilities = ["Maintenance Hub", "Lecture Hall", "Library", "Dorm", "Labs"]
num_facilities = len(facilities)
coordinates = np.array([[0, 0], [2, 3], [-1, 2], [3, -1], [-2, -2]])  # Approximate coordinates
dist_graph = np.array([
    [0, 300, 200, 400, 250],  # Maintenance Hub
    [300, 0, 250, 350, 400],  # Lecture Hall
    [200, 250, 0, 300, 200],  # Library
    [400, 350, 300, 0, 350],  # Dorm
    [250, 400, 200, 350, 0]   # Labs
])
priorities = np.array([0.0, 0.8, 0.3, 0.5, 0.2])  # Priority for each facility (e.g., Lecture Hall has high priority)

Compute visibility

In [ ]:
def compute_visibility(dist_graph, priorities, priority_weight):
    visibility = np.zeros((num_facilities, num_facilities))
    for i in range(num_facilities):
        for j in range(num_facilities):
            if i != j and dist_graph[i, j] != 0:
                visibility[i, j] = 1 / (dist_graph[i, j] * (1 + priority_weight * priorities[j]))
    return visibility

ACO Functions

In [ ]:
def select_next_city(current_facility, visibility, pheromone, alpha, beta):
    phero_alpha = pheromone[current_facility] ** alpha
    vis_beta = visibility[current_facility] ** beta
    combined_feature = phero_alpha * vis_beta
    combined_feature[combined_feature == 0] = 1e-10
    probs = combined_feature / sum(combined_feature)
    cumulative_sum = np.cumsum(probs)
    r = np.random.random_sample()
    next_facility = np.nonzero(cumulative_sum > r)[0][0]
    return next_facility

def update_pheromone(pheromone, solutions, cost, rho, Q):
    pheromone = (1 - rho) * pheromone
    for ant in range(len(solutions)):
        dt = Q / cost[ant]
        for facility_idx in range(num_facilities):
            pheromone[solutions[ant][facility_idx]][solutions[ant][facility_idx + 1]] += dt
    return pheromone

def run_aco(num_ants, num_iters, alpha, beta, rho, priority_weight, Q):
    pheromone_graph = np.ones((num_facilities, num_facilities))
    visibility = compute_visibility(dist_graph, priorities, priority_weight)
    solutions = np.zeros((num_ants, num_facilities + 1), dtype=int)
    best_solution = solutions[0]
    best_length = np.inf
    start_facility = 0
    for _ in range(num_iters):
        for ant in range(num_ants):
            temp_pheromone = pheromone_graph.copy()
            solutions[ant][0] = start_facility
            for facility_idx in range(num_facilities - 1):
                current_facility = solutions[ant][facility_idx]
                temp_pheromone[:, current_facility] = 0
                next_facility = select_next_city(current_facility, visibility, temp_pheromone, alpha, beta)
                solutions[ant][facility_idx + 1] = next_facility
            solutions[ant][-1] = start_facility

        dist = np.zeros(num_ants)
        for ant in range(num_ants):
            for facility_idx in range(num_facilities):
                dist[ant] += dist_graph[solutions[ant][facility_idx]][solutions[ant][facility_idx + 1]]

        pheromone_graph = update_pheromone(pheromone_graph, solutions, dist, rho, Q)

        current_bindex = np.argmin(dist)
        current_blength = dist[current_bindex]
        if current_blength < best_length:
            best_length = current_blength
            best_solution = solutions[current_bindex]

    return best_solution, best_length

Plot function

In [ ]:
def plot_solution(best_solution, best_length):
    plt.clf()
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(coordinates[:, 0], coordinates[:, 1], c='red', s=100)
    for i, (x, y) in enumerate(coordinates):
        ax.text(x + 0.1, y, facilities[i], fontsize=12)

    for i in range(num_facilities):
        start = coordinates[best_solution[i]]
        end = coordinates[best_solution[i + 1]]
        ax.plot([start[0], end[0]], [start[1], end[1]], 'b-')

    ax.set_title(f"Best Route: {best_length:.2f} meters\nPath: {' -> '.join([facilities[i] for i in best_solution])}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    plt.show()


GUI Widgets

In [ ]:
num_ants_slider = IntSlider(min=1, max=20, value=4, description='Num Ants')
num_iters_slider = IntSlider(min=1, max=50, value=10, description='Iterations')
alpha_slider = FloatSlider(min=0.1, max=5.0, value=1.0, description='Alpha')
beta_slider = FloatSlider(min=0.1, max=5.0, value=1.0, description='Beta')
rho_slider = FloatSlider(min=0.1, max=0.9, value=0.5, description='Rho')
priority_weight_slider = FloatSlider(min=0.0, max=2.0, value=1.0, description='Priority Weight')
run_button = Button(description="Run ACO")

output = Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Running ACO...")
        start_time = time.time()
        best_solution, best_length = run_aco(
            num_ants=num_ants_slider.value,
            num_iters=num_iters_slider.value,
            alpha=alpha_slider.value,
            beta=beta_slider.value,
            rho=rho_slider.value,
            priority_weight=priority_weight_slider.value,
            Q=1
        )
        print(f"ACO finished in {time.time() - start_time:.2f} seconds.")
        plot_solution(best_solution, best_length)

run_button.on_click(on_button_clicked)


Display GUI

In [ ]:
display(VBox([
    num_ants_slider, num_iters_slider, alpha_slider, beta_slider,
    rho_slider, priority_weight_slider, run_button, output
]))